# 2. Spark gyakorlat

Új belépési pont: `SparkSession`

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

23/11/09 09:20:28 WARN Utils: Your hostname, jxn-linux resolves to a loopback address: 127.0.1.1; using 10.10.116.13 instead (on interface wlp4s0)
23/11/09 09:20:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 09:20:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/09 09:20:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Adathalmaz betöltése csv fájlból

In [3]:
df = spark.read\
.format("csv")\
.load("dolgozo.csv")

23/11/09 09:20:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


A `show()` függvény megmutatja a `DataFrame` tartalmát, táblázatos formában

Sajnos automatikusan nem ismerte fel a fejlécet

In [3]:
df.show(5)

+----+-----+-----------+------+---------+-------+-------+-----+
| _c0|  _c1|        _c2|   _c3|      _c4|    _c5|    _c6|  _c7|
+----+-----+-----------+------+---------+-------+-------+-----+
|DKOD| DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|
|7839| KING|  PRESIDENT|  0000|81-NOV-17|   5000|      0|   10|
|7698|BLAKE|    MANAGER|  7839|81-MAY-01|   2850|      0|   30|
|7782|CLARK|    MANAGER|  7839|81-JUN-09|   2450|      0|   10|
|7566|JONES|    MANAGER|  7839|81-APR-02|   2975|      0|   20|
+----+-----+-----------+------+---------+-------+-------+-----+
only showing top 5 rows



Belolvasás közben megadhatjuk, hogy az első sor fejlécként kerüljön beovlasásra

Az `inferschema` segítségével automatikusan beállítódnak a típusok

In [4]:
df = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("dolgozo.csv")

df.printSchema()

root
 |-- DKOD: integer (nullable = true)
 |-- DNEV: string (nullable = true)
 |-- FOGLALKOZAS: string (nullable = true)
 |-- FONOKE: integer (nullable = true)
 |-- BELEPES: string (nullable = true)
 |-- FIZETES: integer (nullable = true)
 |-- JUTALEK: integer (nullable = true)
 |-- OAZON: integer (nullable = true)



Saját séma definiálása és használata beolvasáskor

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

dolgozoSchema = StructType([\
    StructField("DKOD",IntegerType()), \
    StructField("DNEV",StringType()), \
    StructField("FOGLALKOZAS",StringType()), \
    StructField("FONOKE",IntegerType()), \
    StructField("BELEPES",StringType()), \
    StructField("FIZETES",IntegerType()), \
    StructField("JUTALEK",IntegerType()), \
    StructField("OAZON",IntegerType()) 
  ])

df = spark.read.format("csv")\
.schema(dolgozoSchema)\
.option('header', True)\
.option('enforceSchema', True)\
.load("dolgozo.csv")

df.printSchema()

root
 |-- DKOD: integer (nullable = true)
 |-- DNEV: string (nullable = true)
 |-- FOGLALKOZAS: string (nullable = true)
 |-- FONOKE: integer (nullable = true)
 |-- BELEPES: string (nullable = true)
 |-- FIZETES: integer (nullable = true)
 |-- JUTALEK: integer (nullable = true)
 |-- OAZON: integer (nullable = true)



Ki tudjuk listázni a `DataFrame`-ünk oszlopait

In [6]:
df.columns

['DKOD',
 'DNEV',
 'FOGLALKOZAS',
 'FONOKE',
 'BELEPES',
 'FIZETES',
 'JUTALEK',
 'OAZON']

`DataFrame`-et nem csak beolvasni lehet, hanem manuálisan létrehozni is

Egyszerű példa:

In [7]:
dSchema = StructType([\
    StructField("DKOD",IntegerType(),True), \
    StructField("DNEV",StringType(),True), \
    StructField("FIZETES",IntegerType(),True)
  ])

rows = [Row(1111, 'MR.BLACK', 1000),
       Row(2222, 'MR.WHITE', 2000),
       Row(3333, 'MR.PINK', 3000)]

dolgozoRDD = spark.sparkContext.parallelize(rows)
dolgozoDF = spark.createDataFrame(dolgozoRDD, dSchema)
dolgozoDF.show()

23/11/07 20:07:36 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 4)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/jxn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.10, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (192.168.1.7 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/jxn/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.10, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.

Oszlopok kiválasztása

In [8]:
df.select('DNEV').show(2)
df.select('DNEV', 'FIZETES').show(2)

+-----+
| DNEV|
+-----+
| KING|
|BLAKE|
+-----+
only showing top 2 rows

+-----+-------+
| DNEV|FIZETES|
+-----+-------+
| KING|   5000|
|BLAKE|   2850|
+-----+-------+
only showing top 2 rows



Kifejezések a `select`-ben (új oszlop létrheozása kifejezés alapján)

In [9]:
df.selectExpr(
"*",
"(FIZETES + JUTALEK) as kereset")\
.show(5)

+----+------+-----------+------+---------+-------+-------+-----+-------+
|DKOD|  DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|kereset|
+----+------+-----------+------+---------+-------+-------+-----+-------+
|7839|  KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|   5000|
|7698| BLAKE|    MANAGER|  7839|81-MAY-01|   2850|      0|   30|   2850|
|7782| CLARK|    MANAGER|  7839|81-JUN-09|   2450|      0|   10|   2450|
|7566| JONES|    MANAGER|  7839|81-APR-02|   2975|      0|   20|   2975|
|7654|MARTIN|   SALESMAN|  7698|81-SEP-28|   1250|   1400|   30|   2650|
+----+------+-----------+------+---------+-------+-------+-----+-------+
only showing top 5 rows



Aggregáció a `select`-ben

In [10]:
df.selectExpr("avg(FIZETES)", "count(distinct(OAZON))").show()

+------------+---------------------+
|avg(FIZETES)|count(DISTINCT OAZON)|
+------------+---------------------+
|   1976.5625|                    3|
+------------+---------------------+



Új oszlop létrehozása és oszlop átnevezése

In [11]:
from pyspark.sql.functions import expr

df.withColumn("kereset", expr("FIZETES + JUTALEK")).show(2)

df.withColumnRenamed("FIZETES", 'HETI_FIZETES').show(2)

+----+-----+-----------+------+---------+-------+-------+-----+-------+
|DKOD| DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|kereset|
+----+-----+-----------+------+---------+-------+-------+-----+-------+
|7839| KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|   5000|
|7698|BLAKE|    MANAGER|  7839|81-MAY-01|   2850|      0|   30|   2850|
+----+-----+-----------+------+---------+-------+-------+-----+-------+
only showing top 2 rows

+----+-----+-----------+------+---------+------------+-------+-----+
|DKOD| DNEV|FOGLALKOZAS|FONOKE|  BELEPES|HETI_FIZETES|JUTALEK|OAZON|
+----+-----+-----------+------+---------+------------+-------+-----+
|7839| KING|  PRESIDENT|     0|81-NOV-17|        5000|      0|   10|
|7698|BLAKE|    MANAGER|  7839|81-MAY-01|        2850|      0|   30|
+----+-----+-----------+------+---------+------------+-------+-----+
only showing top 2 rows



Oszlop törlése

In [12]:
df.drop('FONOKE').show(2)

+----+-----+-----------+---------+-------+-------+-----+
|DKOD| DNEV|FOGLALKOZAS|  BELEPES|FIZETES|JUTALEK|OAZON|
+----+-----+-----------+---------+-------+-------+-----+
|7839| KING|  PRESIDENT|81-NOV-17|   5000|      0|   10|
|7698|BLAKE|    MANAGER|81-MAY-01|   2850|      0|   30|
+----+-----+-----------+---------+-------+-------+-----+
only showing top 2 rows



Sorok szűrése

In [13]:
from pyspark.sql.functions import col

`Filter` kulcsszó használatával:

In [14]:
df.filter(col('DEST') == 'United States').show(4)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `DEST` cannot be resolved. Did you mean one of the following? [`DNEV`, `DKOD`, `BELEPES`, `FIZETES`, `OAZON`].;
'Filter ('DEST = United States)
+- Relation [DKOD#108,DNEV#109,FOGLALKOZAS#110,FONOKE#111,BELEPES#112,FIZETES#113,JUTALEK#114,OAZON#115] csv


`Where` kulcsszó használatával (mindkettő ugyanazt csinálja):

In [15]:
df.where(col('OAZON') == 20).where(col('FIZETES') >= 3000).show()

+----+-----+-----------+------+---------+-------+-------+-----+
|DKOD| DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|
+----+-----+-----------+------+---------+-------+-------+-----+
|7902| FORD|    ANALYST|  7566|81-DEC-03|   3000|      0|   20|
|7788|SCOTT|    ANALYST|  7566|82-DEC-09|   3000|      0|   20|
+----+-----+-----------+------+---------+-------+-------+-----+



Példa a VAGY használatára (ugyanígy működik az & operátor is)

In [16]:
df.where( (col('FONOKE') == 7839) | (col('FIZETES')>3000) ).show()

+----+-----+-----------+------+---------+-------+-------+-----+
|DKOD| DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|
+----+-----+-----------+------+---------+-------+-------+-----+
|7839| KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|
|7698|BLAKE|    MANAGER|  7839|81-MAY-01|   2850|      0|   30|
|7782|CLARK|    MANAGER|  7839|81-JUN-09|   2450|      0|   10|
|7566|JONES|    MANAGER|  7839|81-APR-02|   2975|      0|   20|
+----+-----+-----------+------+---------+-------+-------+-----+



További műveletek

Duplikátumok eltávolítása

In [17]:
df.select(col('OAZON')).distinct().count()

3

Rendezés

In [18]:
df.orderBy(col('OAZON'), col('FIZETES').desc()).show(5)

+----+------+-----------+------+---------+-------+-------+-----+
|DKOD|  DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|
+----+------+-----------+------+---------+-------+-------+-----+
|7839|  KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|
|7782| CLARK|    MANAGER|  7839|81-JUN-09|   2450|      0|   10|
|7934|MILLER|      CLERK|  7782|82-JAN-23|   1300|      0|   10|
|7877|  LOLA|      CLERK|  7902|81-JAN-12|    800|      0|   10|
|7902|  FORD|    ANALYST|  7566|81-DEC-03|   3000|      0|   20|
+----+------+-----------+------+---------+-------+-------+-----+
only showing top 5 rows



Csoportosítás

In [19]:
df.groupBy('OAZON').count().show()

+-----+-----+
|OAZON|count|
+-----+-----+
|   20|    6|
|   10|    4|
|   30|    6|
+-----+-----+



Aggregáció csoportosítás után az `agg()` függvénnyel

Az `agg()` függvénybe más aggregációs függvény is kerülhet: `count`, `sum`, `min`, `max` stb.


In [20]:
df.groupBy('OAZON').agg( countDistinct('DNEV'), sum('FIZETES') ).show()

+-----+--------------------+------------+
|OAZON|count(DISTINCT DNEV)|sum(FIZETES)|
+-----+--------------------+------------+
|   20|                   6|       12675|
|   10|                   4|        9550|
|   30|                   6|        9400|
+-----+--------------------+------------+



Összekapcsolás

Összekapcsoláshoz olvassunk be egy másik `DataFrame`-et

`df.join(df2, join_expression, join_type)`

In [21]:
osztaly = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("osztaly.csv")

osztaly.show()

+-----+----------+---------+
|OAZON|       NEV|TELEPHELY|
+-----+----------+---------+
|   10|ACCOUNTING| NEW YORK|
|   20|  RESEARCH|   DALLAS|
|   30|     SALES|  CHICAGO|
|   40|OPERATIONS|   BOSTON|
+-----+----------+---------+



In [22]:
df.alias('D')\
.join(osztaly.alias('O'), col('D.OAZON') == col('O.OAZON'), 'inner')\
.show(5)

+----+------+-----------+------+---------+-------+-------+-----+-----+----------+---------+
|DKOD|  DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|OAZON|       NEV|TELEPHELY|
+----+------+-----------+------+---------+-------+-------+-----+-----+----------+---------+
|7839|  KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|   10|ACCOUNTING| NEW YORK|
|7698| BLAKE|    MANAGER|  7839|81-MAY-01|   2850|      0|   30|   30|     SALES|  CHICAGO|
|7782| CLARK|    MANAGER|  7839|81-JUN-09|   2450|      0|   10|   10|ACCOUNTING| NEW YORK|
|7566| JONES|    MANAGER|  7839|81-APR-02|   2975|      0|   20|   20|  RESEARCH|   DALLAS|
|7654|MARTIN|   SALESMAN|  7698|81-SEP-28|   1250|   1400|   30|   30|     SALES|  CHICAGO|
+----+------+-----------+------+---------+-------+-------+-----+-----+----------+---------+
only showing top 5 rows



Tábla létrehozás a `DataFrame` alapján - ezután lehetőségünk van a tábla nevére hivatkozni SQL lekérdezéseknél


In [23]:
df.createOrReplaceTempView("dolgozoTable")

SQL lekérdezés írása

In [24]:
spark.sql('SELECT * FROM dolgozoTable').show(1)

+----+----+-----------+------+---------+-------+-------+-----+
|DKOD|DNEV|FOGLALKOZAS|FONOKE|  BELEPES|FIZETES|JUTALEK|OAZON|
+----+----+-----------+------+---------+-------+-------+-----+
|7839|KING|  PRESIDENT|     0|81-NOV-17|   5000|      0|   10|
+----+----+-----------+------+---------+-------+-------+-----+
only showing top 1 row



Komplexebb lekérdezés is írható

In [25]:
spark.sql('''
SELECT OAZON, count(*) 
FROM dolgozoTable 
GROUP BY OAZON
''').show()

+-----+--------+
|OAZON|count(1)|
+-----+--------+
|   20|       6|
|   10|       4|
|   30|       6|
+-----+--------+



Végrehajtási tervek

In [26]:
spark.sql('SELECT OAZON, count(*) FROM dolgozoTable GROUP BY OAZON').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[OAZON#115], functions=[count(1)])
   +- Exchange hashpartitioning(OAZON#115, 200), ENSURE_REQUIREMENTS, [plan_id=633]
      +- HashAggregate(keys=[OAZON#115], functions=[partial_count(1)])
         +- FileScan csv [OAZON#115] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jxn/code/elte/bigdata/spark/spark/spark2/dolgozo.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<OAZON:int>




Ugyanaz a végrehajtási terv a Sparkos megoldás és az SQL-es megoldás során is

In [27]:
df.groupBy('OAZON').count().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[OAZON#115], functions=[count(1)])
   +- Exchange hashpartitioning(OAZON#115, 200), ENSURE_REQUIREMENTS, [plan_id=646]
      +- HashAggregate(keys=[OAZON#115], functions=[partial_count(1)])
         +- FileScan csv [OAZON#115] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jxn/code/elte/bigdata/spark/spark/spark2/dolgozo.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<OAZON:int>




Retail adatok beolvasása

In [28]:
retail = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("online_retail_data.csv")

retail.createOrReplaceTempView("retail_table")
retail.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01/12/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



Kiszámíthatunk alapvető statisztikákat a `DataFrame` oszlopaira

In [29]:
retail.select('UnitPrice', 'Quantity').summary().show()

+-------+-----------------+------------------+
|summary|        UnitPrice|          Quantity|
+-------+-----------------+------------------+
|  count|           542014|            542014|
|   mean|4.611339197881223| 9.555284549845576|
| stddev|96.75097958042586|218.06048544770547|
|    min|        -11062.06|            -80995|
|    25%|             1.25|                 1|
|    50%|             2.08|                 3|
|    75%|             4.13|                10|
|    max|          38970.0|             80995|
+-------+-----------------+------------------+



Melyik országban él a legtöbb vásárló? (Spark)

In [30]:
retail\
.groupBy("Country")\
.agg(countDistinct("CustomerID").alias("cnt"))\
.orderBy(col("cnt").desc())\
.limit(1)\
.show()

+--------------+----+
|       Country| cnt|
+--------------+----+
|United Kingdom|3950|
+--------------+----+



Melyik országban él a legtöbb vásárló? (SQL)

In [ ]:
spark.sql('''
select Country, count(distinct CustomerID) as cnt from retail_table
group by Country
order by cnt desc LIMIT 1 
''').show()

Adjuk meg azt az 5 országot, amelyekből a legtöbb bevétel származik (Spark)

In [ ]:
retail.selectExpr('*', '(Quantity * UnitPrice) as bevetel')\
.groupBy('Country')\
.sum('bevetel')\
.orderBy(col('sum(bevetel)').desc() )\
.show(5)

Adjuk meg azt az 5 országot, amelyekből a legtöbb bevétel származik (SQL)

In [ ]:
spark.sql('''
select Country, sum(Quantity * UnitPrice) 
from retail_table
group by Country
order by sum(Quantity * UnitPrice) desc
''').show(5)

Melyik termék termelte a legnagyobb bevételt? (SQL)

In [ ]:
spark.sql('''
select StockCode, Description, sum(Quantity * UnitPrice) from retail_table
group by StockCode, Description
order by sum(Quantity * UnitPrice) desc
''').show(1)

Melyik a legnépszerűbb termék (amelyikből a legtöbb darabot eladták)? (Spark)

In [ ]:
retail\
.groupBy("StockCode")\
.agg(sum(col("Quantity")).alias("db"))\
.orderBy(desc("db"))\
.limit(1)\
.show()

Melyik a legnépszerűbb termék (amelyikből a legtöbb darabot eladták)? (SQL)

In [ ]:
spark.sql('''
select StockCode, sum(Quantity) from retail_table
group by StockCode
order by sum(Quantity) desc
''').show(1)

Átlagosan hány különböző terméket vesz egy vásárló egy vásárlás során? (Spark)

In [ ]:
retail2 = retail.groupBy("InvoiceNo").agg(count(col("*")).alias("db"))
retail2.select(avg(col("db"))).show()

Átlagosan hány különböző terméket vesz egy vásárló egy vásárlás során? (SQL)

In [31]:
spark.sql('''
select avg(db) from 
(select InvoiceNo, count(*) db from retail_table
group by InvoiceNo)
''').show(5)

+------------------+
|           avg(db)|
+------------------+
|20.927181467181466|
+------------------+



Melyik tranzakció során vásárolták a legtöbb különböző terméket és hányat? (Spark)

In [ ]:
retail\
.groupBy("InvoiceNo")\
.agg(count(col("*")).alias("cnt"))\
.orderBy(desc("cnt"))\
.limit(1)\
.show()

Melyik tranzakció során vásárolták a legtöbb különböző terméket és hányat? (SQL)

In [ ]:
spark.sql('''
select InvoiceNo, count(*) from retail_table
group by InvoiceNo
order by count(*) desc
''').show(1)